In [1]:
import requests
import json
import datetime
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import bands
import fires
import ioexcel
import os
from pathlib import Path
import sqlite3 
 
url_prefix = "https://modis.ornl.gov/rst/api/v1/"
header = {'Accept': 'application/json'}

In [2]:
conn = sqlite3.connect('Data/FPA_FOD_20221014.sqlite')
#in lieu of dataframe manipulation in the next code block:
df = pd.read_sql_query("""SELECT *
                       FROM Fires""", conn)

In [3]:
df.columns

Index(['OBJECTID', 'Shape', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE',
       'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME',
       'ICS_209_PLUS_INCIDENT_JOIN_ID', 'ICS_209_PLUS_COMPLEX_JOIN_ID',
       'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'FIRE_YEAR',
       'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME',
       'NWCG_CAUSE_CLASSIFICATION', 'NWCG_GENERAL_CAUSE',
       'NWCG_CAUSE_AGE_CATEGORY', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME',
       'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_DESCR',
       'STATE', 'COUNTY', 'FIPS_CODE', 'FIPS_NAME'],
      dtype='object')

In [18]:
_df_bottom_n_CA = df.query("STATE=='WY' & FIRE_SIZE_CLASS=='G' & FIRE_YEAR > 2003").sort_values(by='FIRE_SIZE') #[['FIRE_SIZE', 'STATE']].head(30) #FIRE_YEAR> 2004 
_df_bottom_n_CA = _df_bottom_n_CA.head(50)   
_df_bottom_n_CA

,OBJECTID,Shape,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,...,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME
1520873,1520874,b'\x00\x01\xad\x10\x00\x00\\q\x94c\xc2\x15Z\xc...,201216045,ICS209_2011_WY-NIX-011276,INTERAGCY,IA-ICS209,ST/C&L,USWYNIX,Niobrara County,WYNIX,...,None,5000.00,G,43.382778,-104.340000,COUNTY,WY,NIOBRARA,56027,Niobrara County
1597759,1597760,b'\x00\x01\xad\x10\x00\x00t\xb3\xe8\x98\xb1\x1...,201617805,SFO-2012WYWSFD444,NONFED,ST-NASF,ST/C&L,USWYWYS,Wyoming State Forestry,WYWSFD,...,2000,5000.00,G,42.557033,-104.354600,MISSING/NOT SPECIFIED,WY,Goshen,56015,Goshen County
1304784,1304785,b'\x00\x01\xad\x10\x00\x000\x9f\xe3\x0b)\x95Z\...,1779535,SFO-2004WYWSFD2369,NONFED,ST-NASF,ST/C&L,USWYWYS,Wyoming State Forestry,WYWSFD,...,1352,5020.00,G,42.830852,-106.330641,MISSING/NOT SPECIFIED,WY,None,None,None
1217891,1217892,b'\x00\x01\xad\x10\x00\x00p/\x9d\xc7\xc2\xb4[\...,1476536,W-615385,FED,DOI-WFMI,BLM,USWYKRD,Kemmerer Field Office,WYKRD,...,2000,5261.00,G,41.283100,-110.824400,PRIVATE,WY,Uinta,56041,Uinta County
1181820,1181821,b'\x00\x01\xad\x10\x00\x00\xb8\x80\xc7R\x0c\xa...,1434030,FS-1492406,FED,FS-FIRESTAT,FS,USWYBTF,Bridger-Teton National Forest,0403,...,0945,5429.00,G,43.328056,-110.578889,USFS,WY,039,56039,Teton County
1218279,1218280,b'\x00\x01\xad\x10\x00\x00\xdc\x11\xed\xe1\xad...,1476940,W-615177,FED,DOI-WFMI,NPS,USWYYNP,Yellowstone National Park,WYYNP,...,1600,5510.00,G,44.836700,-110.370000,NPS,WY,None,None,None
1456864,1456865,b'\x00\x01\xad\x10\x00\x00D\xf3eA\x7fdZ\xc0P\x...,20024292,FS-1498972,FED,FS-FIRESTAT,FS,USWYMRF,"Medicine Bow-Routt National Forests, Thunder B...",0206,...,2000,5670.00,G,43.338611,-105.570278,STATE OR PRIVATE,WY,None,None,None
1802923,1802924,b'\x00\x01\xad\x10\x00\x00\xe8]\xfb\x83\x9a\x8...,300217510,W-681864,FED,DOI-WFMI,BLM,USWYBFD,Buffalo Field Office,WYBFD,...,1200,5723.60,G,44.950094,-106.150067,PRIVATE,WY,Sheridan,56033,Sheridan County
1580876,1580877,b'\x00\x01\xad\x10\x00\x00`\xfe\x0c\x9av\x88Z\...,201511393,W-645779,FED,DOI-WFMI,BLM,USWYBFD,Buffalo Field Office,WYBFD,...,1300,5779.00,G,44.999110,-106.132250,STATE,WY,Sheridan,56033,Sheridan County
1580901,1580902,b'\x00\x01\xad\x10\x00\x00\xa4\xa3\xee\x93;jZ\...,201511419,W-648432,FED,DOI-WFMI,BLM,USWYCAD,Casper Field Office,WYCAD,...,1200,5882.00,G,42.745900,-105.659897,PRIVATE,WY,Converse,56009,Converse County


In [19]:
_df_bottom_n_CA = _df_bottom_n_CA[['OBJECTID','LATITUDE', 'LONGITUDE', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY',
                'FIRE_SIZE', 'FIRE_SIZE_CLASS' ]]

In [20]:
ioexcel.write_to_excel_fires(_df_bottom_n_CA, "RM2_ClassG")